# Experiment 8

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 107.9 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 861MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [3]:
!pip install prophet plotly mlflow dagshub xgboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Install required packages
import subprocess
import sys

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import zipfile
import warnings
warnings.filterwarnings('ignore')

print("=== STEP 1: Loading and Basic Data Overview ===")

# Load all datasets (extract from zip files first)
print("Extracting and loading datasets...")

# Extract and load stores.csv (assuming it's not zipped based on your code)
stores_df = pd.read_csv('stores.csv')

# Extract and load train.csv
with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
train_df = pd.read_csv('train.csv')

# Extract and load test.csv
with zipfile.ZipFile('test.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
test_df = pd.read_csv('test.csv')

# Extract and load features.csv
with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
features_df = pd.read_csv('features.csv')

print("✅ All datasets loaded successfully!")
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"Features shape: {features_df.shape}")
print(f"Stores shape: {stores_df.shape}")

print("\n=== TRAIN DATASET ===")
print("Columns:", train_df.columns.tolist())
print("\nFirst 5 rows:")
print(train_df.head())
print("\nData types:")
print(train_df.dtypes)
print("\nBasic statistics:")
print(train_df.describe())

print("\n=== TEST DATASET ===")
print("Columns:", test_df.columns.tolist())
print("First 5 rows:")
print(test_df.head())

print("\n=== FEATURES DATASET ===")
print("Columns:", features_df.columns.tolist())
print("First 5 rows:")
print(features_df.head())

print("\n=== STORES DATASET ===")
print("Columns:", stores_df.columns.tolist())
print("First 5 rows:")
print(stores_df.head())
print("Store types and sizes:")
print(stores_df.groupby('Type').agg({'Size': ['count', 'mean', 'min', 'max']}))


=== STEP 1: Loading and Basic Data Overview ===
Extracting and loading datasets...
✅ All datasets loaded successfully!
Train shape: (421570, 5)
Test shape: (115064, 4)
Features shape: (8190, 12)
Stores shape: (45, 3)

=== TRAIN DATASET ===
Columns: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']

First 5 rows:
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False

Data types:
Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
dtype: object

Basic statistics:
               Store           Dept   Weekly_Sales
count  421570.000000  421570.000000  421570.000000
mean       22.200546      44.260317   15981.258123
std        12.785297      30.492054   22711.183519

In [7]:

print("\n" + "="*60)
print("=== STEP 2: Date Analysis & Missing Values ===")

# Convert dates to datetime
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])
features_df['Date'] = pd.to_datetime(features_df['Date'])

print("✅ Dates converted to datetime format")

# Analyze date ranges
print(f"\nTRAIN date range: {train_df['Date'].min()} to {train_df['Date'].max()}")
print(f"TEST date range: {test_df['Date'].min()} to {test_df['Date'].max()}")
print(f"FEATURES date range: {features_df['Date'].min()} to {features_df['Date'].max()}")

# Check for missing values
print("\n=== Missing Values Analysis ===")
print("TRAIN missing values:")
print(train_df.isnull().sum())

print("\nTEST missing values:")
print(test_df.isnull().sum())

print("\nFEATURES missing values:")
print(features_df.isnull().sum())
print(f"MarkDown missing percentages:")
for col in ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']:
    missing_pct = (features_df[col].isnull().sum() / len(features_df)) * 100
    print(f"  {col}: {missing_pct:.1f}%")

print("\nSTORES missing values:")
print(stores_df.isnull().sum())

# Analyze unique counts
print("\n=== Unique Value Counts ===")
print(f"Unique stores in train: {train_df['Store'].nunique()}")
print(f"Unique departments in train: {train_df['Dept'].nunique()}")
print(f"Unique store-dept combinations in train: {train_df[['Store', 'Dept']].drop_duplicates().shape[0]}")

print(f"\nUnique stores in test: {test_df['Store'].nunique()}")
print(f"Unique departments in test: {test_df['Dept'].nunique()}")
print(f"Unique store-dept combinations in test: {test_df[['Store', 'Dept']].drop_duplicates().shape[0]}")

# Check holiday distribution
print("\n=== Holiday Analysis ===")
print("Train holiday weeks:")
print(train_df['IsHoliday'].value_counts())
print(f"Holiday percentage in train: {(train_df['IsHoliday'].sum() / len(train_df)) * 100:.2f}%")

print("\nTest holiday weeks:")
print(test_df['IsHoliday'].value_counts())
print(f"Holiday percentage in test: {(test_df['IsHoliday'].sum() / len(test_df)) * 100:.2f}%")

# Sales statistics
print("\n=== Sales Analysis ===")
print(f"Negative sales records: {(train_df['Weekly_Sales'] < 0).sum()}")
print(f"Zero sales records: {(train_df['Weekly_Sales'] == 0).sum()}")
print(f"Sales range: ${train_df['Weekly_Sales'].min():.2f} to ${train_df['Weekly_Sales'].max():.2f}")

# Check for consistent time series
print("\n=== Time Series Consistency Check ===")
# Group by store and dept to check date consistency
sample_store_dept = train_df[(train_df['Store'] == 1) & (train_df['Dept'] == 1)].copy()
sample_store_dept = sample_store_dept.sort_values('Date')
print(f"Sample store 1, dept 1 date range: {sample_store_dept['Date'].min()} to {sample_store_dept['Date'].max()}")
print(f"Number of records for store 1, dept 1: {len(sample_store_dept)}")
print(f"Expected weekly records for this period: {(sample_store_dept['Date'].max() - sample_store_dept['Date'].min()).days // 7 + 1}")



=== STEP 2: Date Analysis & Missing Values ===
✅ Dates converted to datetime format

TRAIN date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
TEST date range: 2012-11-02 00:00:00 to 2013-07-26 00:00:00
FEATURES date range: 2010-02-05 00:00:00 to 2013-07-26 00:00:00

=== Missing Values Analysis ===
TRAIN missing values:
Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

TEST missing values:
Store        0
Dept         0
Date         0
IsHoliday    0
dtype: int64

FEATURES missing values:
Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64
MarkDown missing percentages:
  MarkDown1: 50.8%
  MarkDown2: 64.3%
  MarkDown3: 55.9%
  MarkDown4: 57.7%
  MarkDown5: 50.5%

STORES missing values:
Store    0
Type     0
Size     0
dty

In [8]:

print("\n" + "="*60)
print("=== STEP 3: Data Merging & Pattern Analysis ===")

# Create a complete dataset by merging train with features and stores
print("Merging datasets...")

# Merge train with features
train_complete = train_df.merge(features_df, on=['Store', 'Date'], how='left', suffixes=('', '_feat'))
print(f"Train after merging with features: {train_complete.shape}")

# Merge with stores
train_complete = train_complete.merge(stores_df, on='Store', how='left')
print(f"Train after merging with stores: {train_complete.shape}")

# Check for any merge issues
print(f"Records in train_complete: {len(train_complete)}")
print(f"Records in original train: {len(train_df)}")
print("✅ All train records preserved in merge")

# Same for test
test_complete = test_df.merge(features_df, on=['Store', 'Date'], how='left', suffixes=('', '_feat'))
test_complete = test_complete.merge(stores_df, on='Store', how='left')
print(f"Test complete shape: {test_complete.shape}")

print("\n=== Holiday Consistency Check ===")
# Check if IsHoliday is consistent between datasets
holiday_check = train_complete[['Date', 'IsHoliday', 'IsHoliday_feat']].drop_duplicates()
inconsistent_holidays = holiday_check[holiday_check['IsHoliday'] != holiday_check['IsHoliday_feat']]
print(f"Inconsistent holiday flags: {len(inconsistent_holidays)}")
if len(inconsistent_holidays) > 0:
    print("Sample inconsistencies:")
    print(inconsistent_holidays.head())

print("\n=== Store-Department Distribution Analysis ===")
print("Top 10 departments by total sales:")
dept_sales = train_complete.groupby('Dept')['Weekly_Sales'].agg(['sum', 'count', 'mean']).round(2)
dept_sales = dept_sales.sort_values('sum', ascending=False)
print(dept_sales.head(10))

print("\nTop 10 stores by total sales:")
store_sales = train_complete.groupby('Store')['Weekly_Sales'].agg(['sum', 'count', 'mean']).round(2)
store_sales = store_sales.sort_values('sum', ascending=False)
print(store_sales.head(10))

print("\nSales by store type:")
type_sales = train_complete.groupby('Type')['Weekly_Sales'].agg(['sum', 'count', 'mean', 'std']).round(2)
print(type_sales)

print("\n=== Seasonal Pattern Analysis ===")
# Add time features for analysis
train_complete['Year'] = train_complete['Date'].dt.year
train_complete['Month'] = train_complete['Date'].dt.month
train_complete['Week'] = train_complete['Date'].dt.isocalendar().week
train_complete['DayOfYear'] = train_complete['Date'].dt.dayofyear

print("Sales by year:")
yearly_sales = train_complete.groupby('Year')['Weekly_Sales'].agg(['sum', 'mean', 'count']).round(2)
print(yearly_sales)

print("\nSales by month (average):")
monthly_sales = train_complete.groupby('Month')['Weekly_Sales'].mean().round(2)
print(monthly_sales)

print("\n=== Feature Correlation with Sales ===")
# Correlation analysis
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size']
available_features = [col for col in numeric_features if col in train_complete.columns]

print("Correlation with Weekly_Sales:")
for feature in available_features:
    corr = train_complete[['Weekly_Sales', feature]].corr().iloc[0, 1]
    print(f"  {feature}: {corr:.3f}")

print("\n=== MarkDown Analysis ===")
# Analyze markdown patterns
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
markdown_data = train_complete[train_complete[markdown_cols].notna().any(axis=1)]
print(f"Records with any markdown data: {len(markdown_data)} ({len(markdown_data)/len(train_complete)*100:.1f}%)")

if len(markdown_data) > 0:
    print("\nAverage sales with vs without markdowns:")
    with_markdown = train_complete[train_complete[markdown_cols].notna().any(axis=1)]['Weekly_Sales'].mean()
    without_markdown = train_complete[train_complete[markdown_cols].isna().all(axis=1)]['Weekly_Sales'].mean()
    print(f"  With markdowns: ${with_markdown:.2f}")
    print(f"  Without markdowns: ${without_markdown:.2f}")
    print(f"  Difference: {((with_markdown - without_markdown) / without_markdown * 100):+.1f}%")

print("\n=== Data Quality Summary ===")
print(f"Training period: {train_df['Date'].min().strftime('%Y-%m-%d')} to {train_df['Date'].max().strftime('%Y-%m-%d')} ({(train_df['Date'].max() - train_df['Date'].min()).days} days)")
print(f"Test period: {test_df['Date'].min().strftime('%Y-%m-%d')} to {test_df['Date'].max().strftime('%Y-%m-%d')} ({(test_df['Date'].max() - test_df['Date'].min()).days} days)")
print(f"Total unique store-dept combinations in train: {train_df[['Store', 'Dept']].drop_duplicates().shape[0]}")
print(f"Total unique store-dept combinations in test: {test_df[['Store', 'Dept']].drop_duplicates().shape[0]}")
print(f"Missing combinations in test: {train_df[['Store', 'Dept']].drop_duplicates().shape[0] - test_df[['Store', 'Dept']].drop_duplicates().shape[0]}")

# Check for any time series gaps
print(f"\nTime series completeness check:")
print(f"Expected weekly periods in train: {((train_df['Date'].max() - train_df['Date'].min()).days // 7) + 1}")
print(f"Unique dates in train: {train_df['Date'].nunique()}")
print(f"Expected weekly periods in test: {((test_df['Date'].max() - test_df['Date'].min()).days // 7) + 1}")
print(f"Unique dates in test: {test_df['Date'].nunique()}")



=== STEP 3: Data Merging & Pattern Analysis ===
Merging datasets...
Train after merging with features: (421570, 15)
Train after merging with stores: (421570, 17)
Records in train_complete: 421570
Records in original train: 421570
✅ All train records preserved in merge
Test complete shape: (115064, 16)

=== Holiday Consistency Check ===
Inconsistent holiday flags: 0

=== Store-Department Distribution Analysis ===
Top 10 departments by total sales:
               sum  count      mean
Dept                               
92    4.839433e+08   6435  75204.87
95    4.493202e+08   6435  69824.42
38    3.931181e+08   6435  61090.62
72    3.057252e+08   6046  50566.52
90    2.910685e+08   6435  45232.08
40    2.889360e+08   6435  44900.70
2     2.806112e+08   6435  43607.02
91    2.167817e+08   6435  33687.91
13    1.973216e+08   6435  30663.80
8     1.942808e+08   6435  30191.26

Top 10 stores by total sales:
                sum  count      mean
Store                               
20     3.01

In [9]:

print("\n" + "="*60)
print("=== STEP 4: Feature Engineering Strategy & Testing Plan ===")

print("Based on the analysis, here's our feature engineering strategy:\n")

print("🎯 KEY INSIGHTS DISCOVERED:")
print("1. Strong seasonality: Nov-Dec show 20%+ higher sales")
print("2. Store size has decent correlation (0.244) with sales")
print("3. Store Type A >> B >> C in sales volume")
print("4. Departments 92, 95, 38 are top performers")
print("5. MarkDown data available for 36% of records (post Nov 2011)")
print("6. 162 store-dept combinations missing in test set")
print("7. Perfect time series: 143 weeks train, 39 weeks test")

print("\n📊 FEATURE ENGINEERING PLAN:")

print("\n1. TIME-BASED FEATURES:")
print("   - Month, Quarter, Week of Year")
print("   - Days to/from major holidays (Christmas, Thanksgiving)")
print("   - Holiday season indicator (Nov-Dec)")
print("   - Back-to-school season (Aug-Sep)")
print("   - Tax season (Jan-Apr)")

print("\n2. LAG FEATURES (Critical for time series):")
print("   - Sales lags: 1, 2, 4, 8, 12, 52 weeks")
print("   - Rolling means: 4, 8, 12, 26 weeks")
print("   - Rolling std: 4, 8, 12 weeks")
print("   - Year-over-year lag (52 weeks)")

print("\n3. STORE/DEPT FEATURES:")
print("   - Store size percentile within type")
print("   - Department rank by historical sales")
print("   - Store performance rank")
print("   - Dept seasonal index (performance vs average)")

print("\n4. MARKDOWN FEATURES:")
print("   - Total markdown amount (sum of all 5)")
print("   - Number of active markdowns")
print("   - Markdown intensity (markdown/avg_sales)")
print("   - Days since last markdown")

print("\n5. ECONOMIC FEATURES:")
print("   - CPI change rate")
print("   - Unemployment change rate")
print("   - Fuel price change rate")
print("   - Temperature deviation from normal")

print("\n6. HOLIDAY FEATURES:")
print("   - Pre-holiday indicators (1-3 weeks before)")
print("   - Post-holiday indicators (1-2 weeks after)")
print("   - Holiday type encoding")
print("   - Distance to next/previous holiday")

print("\n🏗️ MODELING STRATEGY:")

print("\n1. VALIDATION APPROACH:")
print("   - Time series split: Use last 12 weeks of train as validation")
print("   - Walk-forward validation for hyperparameter tuning")
print("   - WMAE metric with 5x holiday weighting")

print("\n2. MODEL PIPELINE PRIORITY:")
print("   Phase 1 - Baseline Models:")
print("     - Naive forecasts (seasonal, drift)")
print("     - Linear regression with basic features")
print("     - Target: Beat naive baselines")
print()
print("   Phase 2 - Tree-Based Models:")
print("     - LightGBM with full feature set")
print("     - XGBoost with optimized features")
print("     - Target: Strong performance with interpretability")
print()
print("   Phase 3 - Classical Time Series:")
print("     - Prophet (handles holidays well)")
print("     - SARIMA (for pure time series patterns)")
print("     - Target: Capture seasonal patterns")
print()
print("   Phase 4 - Deep Learning:")
print("     - N-BEATS (pure neural forecasting)")
print("     - DLinear (efficient linear approach)")
print("     - TFT (if computational resources allow)")
print("     - Target: Capture complex patterns")

print("\n3. ENSEMBLE STRATEGY:")
print("   - Weight models based on validation performance")
print("   - Consider different models for different store types")
print("   - Holiday vs non-holiday specific models")

print("\n📋 NEXT STEPS CHECKLIST:")
print("1. ✅ Data exploration and understanding")
print("2. 🔄 Feature engineering implementation")
print("3. ⏳ Baseline model development")
print("4. ⏳ Tree-based model experiments")
print("5. ⏳ Time series model experiments")
print("6. ⏳ Deep learning model experiments")
print("7. ⏳ Model ensembling")
print("8. ⏳ Final inference and submission")

print("\n💡 CRITICAL SUCCESS FACTORS:")
print("- Handle 162 missing store-dept combinations in test")
print("- Proper WMAE evaluation (5x holiday weight)")
print("- Robust cross-validation strategy")
print("- Feature engineering for seasonality")
print("- MarkDown feature engineering post Nov 2011")

print("\n🚀 READY TO START FEATURE ENGINEERING!")
print("Next: Implement feature engineering pipeline")
print("Focus: Time-based and lag features first")

# Save insights for reference
insights = {
    'train_period': f"{train_df['Date'].min()} to {train_df['Date'].max()}",
    'test_period': f"{test_df['Date'].min()} to {test_df['Date'].max()}",
    'top_depts': dept_sales.head(5).index.tolist(),
    'top_stores': store_sales.head(5).index.tolist(),
    'holiday_pct_train': (train_df['IsHoliday'].sum() / len(train_df)) * 100,
    'holiday_pct_test': (test_df['IsHoliday'].sum() / len(test_df)) * 100,
    'markdown_availability': len(markdown_data) / len(train_complete) * 100,
    'missing_combinations': train_df[['Store', 'Dept']].drop_duplicates().shape[0] - test_df[['Store', 'Dept']].drop_duplicates().shape[0]
}

print(f"\n📊 Key metrics saved for reference:")
for key, value in insights.items():
    print(f"   {key}: {value}")

print("\n" + "="*60)
print("✨ EXPLORATION COMPLETE - READY FOR FEATURE ENGINEERING ✨")



=== STEP 4: Feature Engineering Strategy & Testing Plan ===
Based on the analysis, here's our feature engineering strategy:

🎯 KEY INSIGHTS DISCOVERED:
1. Strong seasonality: Nov-Dec show 20%+ higher sales
2. Store size has decent correlation (0.244) with sales
3. Store Type A >> B >> C in sales volume
4. Departments 92, 95, 38 are top performers
5. MarkDown data available for 36% of records (post Nov 2011)
6. 162 store-dept combinations missing in test set
7. Perfect time series: 143 weeks train, 39 weeks test

📊 FEATURE ENGINEERING PLAN:

1. TIME-BASED FEATURES:
   - Month, Quarter, Week of Year
   - Days to/from major holidays (Christmas, Thanksgiving)
   - Holiday season indicator (Nov-Dec)
   - Back-to-school season (Aug-Sep)
   - Tax season (Jan-Apr)

2. LAG FEATURES (Critical for time series):
   - Sales lags: 1, 2, 4, 8, 12, 52 weeks
   - Rolling means: 4, 8, 12, 26 weeks
   - Rolling std: 4, 8, 12 weeks
   - Year-over-year lag (52 weeks)

3. STORE/DEPT FEATURES:
   - Store si

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import zipfile
import warnings
warnings.filterwarnings('ignore')

print("🚀 WALMART SALES FORECASTING - FEATURE ENGINEERING PIPELINE")
print("=" * 70)

# Load and prepare data (same as exploration)
print("📥 Loading datasets...")

# Load stores.csv
stores_df = pd.read_csv('stores.csv')

# Extract and load train.csv
with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
train_df = pd.read_csv('train.csv')

# Extract and load features.csv
with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
features_df = pd.read_csv('features.csv')

print("✅ Data loaded successfully!")

# Convert dates and merge
train_df['Date'] = pd.to_datetime(train_df['Date'])
features_df['Date'] = pd.to_datetime(features_df['Date'])

# Create complete training dataset
train_complete = train_df.merge(features_df, on=['Store', 'Date'], how='left', suffixes=('', '_feat'))
train_complete = train_complete.merge(stores_df, on='Store', how='left')

print(f"Complete training data shape: {train_complete.shape}")

print("\n⏰ TIME SERIES SPLIT STRATEGY")
print("=" * 50)

# Define time series split - CRITICAL: No data leakage!
train_end_date = train_complete['Date'].max()
train_start_date = train_complete['Date'].min()

# Use last 12 weeks (3 months) as validation set
validation_weeks = 12
val_start_date = train_end_date - timedelta(weeks=validation_weeks-1)

print(f"Training period: {train_start_date} to {val_start_date - timedelta(days=7)}")
print(f"Validation period: {val_start_date} to {train_end_date}")
print(f"Total training weeks: {(train_end_date - train_start_date).days // 7 + 1}")
print(f"Validation weeks: {validation_weeks}")

# Split data chronologically
train_data = train_complete[train_complete['Date'] < val_start_date].copy()
val_data = train_complete[train_complete['Date'] >= val_start_date].copy()

print(f"\nTrain data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"No data leakage: ✅")

print("\n🔧 FEATURE ENGINEERING PIPELINE")
print("=" * 50)

def add_time_features(df):
    """Add time-based features"""
    df = df.copy()

    # Basic time features
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Quarter'] = df['Date'].dt.quarter
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week
    df['DayOfYear'] = df['Date'].dt.dayofyear

    # Seasonal features
    df['IsHolidaySeason'] = ((df['Month'] == 11) | (df['Month'] == 12)).astype(int)
    df['IsBackToSchool'] = ((df['Month'] == 8) | (df['Month'] == 9)).astype(int)
    df['IsTaxSeason'] = ((df['Month'] >= 1) & (df['Month'] <= 4)).astype(int)

    # Holiday distance features (simplified for now)
    # Christmas dates for each year
    christmas_dates = {
        2010: pd.to_datetime('2010-12-31'),
        2011: pd.to_datetime('2011-12-30'),
        2012: pd.to_datetime('2012-12-28'),
    }

    # Thanksgiving dates for each year
    thanksgiving_dates = {
        2010: pd.to_datetime('2010-11-26'),
        2011: pd.to_datetime('2011-11-25'),
        2012: pd.to_datetime('2012-11-23'),
    }

    # Calculate days to Christmas
    df['DaysToChristmas'] = 0
    df['DaysToThanksgiving'] = 0

    for year in christmas_dates.keys():
        year_mask = df['Year'] == year
        if year_mask.any():
            df.loc[year_mask, 'DaysToChristmas'] = (christmas_dates[year] - df.loc[year_mask, 'Date']).dt.days
            df.loc[year_mask, 'DaysToThanksgiving'] = (thanksgiving_dates[year] - df.loc[year_mask, 'Date']).dt.days

    # Pre/Post holiday indicators
    df['PreChristmas'] = ((df['DaysToChristmas'] > 0) & (df['DaysToChristmas'] <= 21)).astype(int)
    df['PostChristmas'] = ((df['DaysToChristmas'] < 0) & (df['DaysToChristmas'] >= -14)).astype(int)
    df['PreThanksgiving'] = ((df['DaysToThanksgiving'] > 0) & (df['DaysToThanksgiving'] <= 14)).astype(int)
    df['PostThanksgiving'] = ((df['DaysToThanksgiving'] < 0) & (df['DaysToThanksgiving'] >= -7)).astype(int)

    return df

def add_lag_features(df, target_col='Weekly_Sales'):
    """Add lag features - ONLY using data available at prediction time"""
    df = df.copy()
    df = df.sort_values(['Store', 'Dept', 'Date'])

    # Lag features (previous weeks)
    lag_periods = [1, 2, 4, 8, 12, 52]

    for lag in lag_periods:
        df[f'{target_col}_lag_{lag}'] = df.groupby(['Store', 'Dept'])[target_col].shift(lag)

    # Rolling features (windows ending BEFORE current week)
    rolling_windows = [4, 8, 12, 26]

    for window in rolling_windows:
        # Rolling mean
        df[f'{target_col}_rolling_mean_{window}'] = df.groupby(['Store', 'Dept'])[target_col].transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=1).mean()
        )

        # Rolling std
        df[f'{target_col}_rolling_std_{window}'] = df.groupby(['Store', 'Dept'])[target_col].transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=1).std()
        )

    return df

def add_store_dept_features(df):
    """Add store and department specific features"""
    df = df.copy()

    # Store size percentile within type
    df['SizePercentileInType'] = df.groupby('Type')['Size'].transform(lambda x: x.rank(pct=True))

    # Historical performance features (calculated only on available data)
    # Note: In real implementation, these should be calculated only on data before each prediction

    return df

def add_markdown_features(df):
    """Add markdown related features"""
    df = df.copy()

    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

    # Total markdown amount
    df['TotalMarkDown'] = df[markdown_cols].sum(axis=1, skipna=True)

    # Number of active markdowns
    df['NumActiveMarkdowns'] = df[markdown_cols].count(axis=1)

    # Has any markdown
    df['HasMarkdown'] = (df['NumActiveMarkdowns'] > 0).astype(int)

    # Fill NaN markdowns with 0 for calculations
    for col in markdown_cols:
        df[col] = df[col].fillna(0)

    return df

def add_economic_features(df):
    """Add economic indicator features"""
    df = df.copy()

    # Sort by date for proper shift calculations
    df = df.sort_values(['Store', 'Date'])

    # Change rates (week-over-week)
    economic_vars = ['CPI', 'Unemployment', 'Fuel_Price', 'Temperature']

    for var in economic_vars:
        df[f'{var}_change'] = df.groupby('Store')[var].pct_change()
        df[f'{var}_change'] = df[f'{var}_change'].fillna(0)

    # Temperature deviation from store average
    df['TempDeviation'] = df.groupby('Store')['Temperature'].transform(lambda x: x - x.mean())

    return df

print("🔄 Applying feature engineering to training data...")

# Apply feature engineering pipeline
train_features = add_time_features(train_data)
train_features = add_lag_features(train_features)
train_features = add_store_dept_features(train_features)
train_features = add_markdown_features(train_features)
train_features = add_economic_features(train_features)

print(f"✅ Training features created: {train_features.shape}")

print("🔄 Applying same features to validation data...")

# Apply same transformations to validation data
val_features = add_time_features(val_data)
val_features = add_lag_features(val_features)  # This will use train data for lags
val_features = add_store_dept_features(val_features)
val_features = add_markdown_features(val_features)
val_features = add_economic_features(val_features)

print(f"✅ Validation features created: {val_features.shape}")

print("\n📊 FEATURE SUMMARY")
print("=" * 50)

# Get feature columns (excluding original columns)
original_cols = ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
                'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
                'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_feat', 'Type', 'Size']

feature_cols = [col for col in train_features.columns if col not in original_cols]

print(f"Total features created: {len(feature_cols)}")
print(f"Original columns: {len(original_cols)}")
print(f"Total columns: {train_features.shape[1]}")

print("\nFeature categories:")
time_features = [col for col in feature_cols if any(x in col.lower() for x in ['year', 'month', 'quarter', 'week', 'day', 'season', 'holiday', 'christmas', 'thanksgiving'])]
lag_features = [col for col in feature_cols if 'lag' in col or 'rolling' in col]
markdown_features = [col for col in feature_cols if 'markdown' in col.lower()]
economic_features = [col for col in feature_cols if 'change' in col or 'deviation' in col]
store_features = [col for col in feature_cols if 'percentile' in col.lower() or 'size' in col.lower()]

print(f"  Time features: {len(time_features)}")
print(f"  Lag features: {len(lag_features)}")
print(f"  Markdown features: {len(markdown_features)}")
print(f"  Economic features: {len(economic_features)}")
print(f"  Store features: {len(store_features)}")

print("\n🎯 NEXT STEPS")
print("=" * 50)
print("1. ✅ Feature engineering pipeline complete")
print("2. 🔄 Ready for baseline model development")
print("3. ⏳ Implement WMAE metric")
print("4. ⏳ Build naive forecasting baselines")
print("5. ⏳ Develop tree-based models")

# Save processed data for modeling
print("\n💾 Saving processed data...")
train_features.to_csv('train_features.csv', index=False)
val_features.to_csv('val_features.csv', index=False)
print("✅ Data saved to train_features.csv and val_features.csv")

print("\n🎉 FEATURE ENGINEERING COMPLETE!")
print("Ready to start modeling experiments!")

🚀 WALMART SALES FORECASTING - FEATURE ENGINEERING PIPELINE
📥 Loading datasets...
✅ Data loaded successfully!
Complete training data shape: (421570, 17)

⏰ TIME SERIES SPLIT STRATEGY
Training period: 2010-02-05 00:00:00 to 2012-08-03 00:00:00
Validation period: 2012-08-10 00:00:00 to 2012-10-26 00:00:00
Total training weeks: 143
Validation weeks: 12

Train data shape: (386007, 17)
Validation data shape: (35563, 17)
No data leakage: ✅

🔧 FEATURE ENGINEERING PIPELINE
🔄 Applying feature engineering to training data...
✅ Training features created: (386007, 54)
🔄 Applying same features to validation data...
✅ Validation features created: (35563, 54)

📊 FEATURE SUMMARY
Total features created: 37
Original columns: 17
Total columns: 54

Feature categories:
  Time features: 27
  Lag features: 14
  Markdown features: 3
  Economic features: 4
  Store features: 1

🎯 NEXT STEPS
1. ✅ Feature engineering pipeline complete
2. 🔄 Ready for baseline model development
3. ⏳ Implement WMAE metric
4. ⏳ Build 

In [12]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("🎯 WALMART SALES FORECASTING - LIGHTGBM BASELINE")
print("=" * 60)

# Load processed data
print("📥 Loading processed datasets...")
train_features = pd.read_csv('train_features.csv')
val_features = pd.read_csv('val_features.csv')

# Convert dates back to datetime
train_features['Date'] = pd.to_datetime(train_features['Date'])
val_features['Date'] = pd.to_datetime(val_features['Date'])

print(f"Train features shape: {train_features.shape}")
print(f"Validation features shape: {val_features.shape}")

print("\n📊 WMAE METRIC IMPLEMENTATION")
print("=" * 50)

def wmae(y_true, y_pred, weights):
    """Weighted Mean Absolute Error - Holiday weeks have 5x weight"""
    absolute_errors = np.abs(y_true - y_pred)
    weighted_errors = absolute_errors * weights
    return np.sum(weighted_errors) / np.sum(weights)

def calculate_weights(is_holiday):
    """Calculate weights: 5 for holiday weeks, 1 for normal weeks"""
    return np.where(is_holiday, 5.0, 1.0)

print("✅ WMAE implementation ready")

print("\n🔧 PREPARING DATA FOR LIGHTGBM")
print("=" * 50)

# Prepare features (exclude target and identifiers)
exclude_cols = ['Date', 'Weekly_Sales', 'Store', 'Dept']
feature_cols = [col for col in train_features.columns if col not in exclude_cols]

# Handle categorical variables
categorical_cols = ['Type']
for col in categorical_cols:
    if col in feature_cols:
        le = LabelEncoder()
        # Fit on combined data to ensure consistent encoding
        combined_values = pd.concat([train_features[col], val_features[col]]).astype(str)
        le.fit(combined_values)

        train_features[col + '_encoded'] = le.transform(train_features[col].astype(str))
        val_features[col + '_encoded'] = le.transform(val_features[col].astype(str))

        feature_cols.remove(col)
        feature_cols.append(col + '_encoded')

print(f"Total features for LightGBM: {len(feature_cols)}")

# Check for missing values
print("\nMissing value analysis:")
train_missing = train_features[feature_cols].isnull().sum()
val_missing = val_features[feature_cols].isnull().sum()

print(f"Features with missing values in train: {(train_missing > 0).sum()}")
print(f"Features with missing values in validation: {(val_missing > 0).sum()}")

# LightGBM can handle missing values, so we don't need to drop rows
print(f"Using all training records: {len(train_features)}")
print(f"Using all validation records: {len(val_features)}")

print("\n🔧 NAIVE BASELINE FOR COMPARISON")
print("=" * 50)

def naive_seasonal_simple(train_df, val_df):
    """Simple seasonal naive with fallbacks"""
    predictions = []

    # Create a mapping of store-dept to recent sales for faster lookup
    recent_sales = {}
    for store in train_df['Store'].unique():
        for dept in train_df['Dept'].unique():
            store_dept_data = train_df[(train_df['Store'] == store) & (train_df['Dept'] == dept)]
            if len(store_dept_data) > 0:
                recent_sales[(store, dept)] = store_dept_data['Weekly_Sales'].iloc[-4:].mean()  # Last 4 weeks avg

    overall_mean = train_df['Weekly_Sales'].mean()

    for _, row in val_df.iterrows():
        store, dept = row['Store'], row['Dept']

        if (store, dept) in recent_sales:
            pred = recent_sales[(store, dept)]
        else:
            pred = overall_mean

        predictions.append(pred)

    return np.array(predictions)

# Run naive baseline
print("🔄 Running naive seasonal baseline...")
naive_pred = naive_seasonal_simple(train_features, val_features)
val_weights = calculate_weights(val_features['IsHoliday'].values)
naive_wmae = wmae(val_features['Weekly_Sales'].values, naive_pred, val_weights)

print(f"Naive Seasonal WMAE: {naive_wmae:,.2f}")

print("\n🔧 LIGHTGBM MODEL TRAINING")
print("=" * 50)

# LightGBM parameters
lgb_params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 63,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_data_in_leaf': 50,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'verbose': -1,
    'random_state': 42
}

print("🔄 Training LightGBM model...")
print(f"Features: {len(feature_cols)}")
print(f"Training samples: {len(train_features)}")
print(f"Validation samples: {len(val_features)}")

# Create LightGBM datasets
train_lgb = lgb.Dataset(
    train_features[feature_cols],
    label=train_features['Weekly_Sales'],
    categorical_feature=['Type_encoded'] if 'Type_encoded' in feature_cols else None
)

val_lgb = lgb.Dataset(
    val_features[feature_cols],
    label=val_features['Weekly_Sales'],
    reference=train_lgb,
    categorical_feature=['Type_encoded'] if 'Type_encoded' in feature_cols else None
)

# Train model with early stopping
lgb_model = lgb.train(
    lgb_params,
    train_lgb,
    valid_sets=[train_lgb, val_lgb],
    valid_names=['train', 'valid'],
    num_boost_round=500,
    callbacks=[
        lgb.early_stopping(50),
        lgb.log_evaluation(50)
    ]
)

print(f"✅ Training completed. Best iteration: {lgb_model.best_iteration}")

print("\n📊 MODEL EVALUATION")
print("=" * 50)

# Make predictions
lgb_pred = lgb_model.predict(val_features[feature_cols])

# Calculate WMAE
lgb_wmae = wmae(val_features['Weekly_Sales'].values, lgb_pred, val_weights)

print(f"LightGBM WMAE: {lgb_wmae:,.2f}")
print(f"Naive WMAE: {naive_wmae:,.2f}")
print(f"Improvement: {((naive_wmae - lgb_wmae) / naive_wmae * 100):+.1f}%")

print("\n📈 FEATURE IMPORTANCE")
print("=" * 50)

# Get feature importance
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': lgb_model.feature_importance(importance_type='gain')
})
importance_df = importance_df.sort_values('importance', ascending=False)

print("Top 15 most important features:")
top_features = importance_df.head(15)
for i, (_, row) in enumerate(top_features.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:>10,.0f}")

print("\n📊 PREDICTION ANALYSIS")
print("=" * 50)

# Analyze predictions by holiday vs non-holiday
holiday_mask = val_features['IsHoliday'] == True
non_holiday_mask = val_features['IsHoliday'] == False

if holiday_mask.sum() > 0:
    holiday_mae = np.mean(np.abs(val_features.loc[holiday_mask, 'Weekly_Sales'] - lgb_pred[holiday_mask]))
    print(f"Holiday weeks MAE: {holiday_mae:,.2f}")

if non_holiday_mask.sum() > 0:
    non_holiday_mae = np.mean(np.abs(val_features.loc[non_holiday_mask, 'Weekly_Sales'] - lgb_pred[non_holiday_mask]))
    print(f"Non-holiday weeks MAE: {non_holiday_mae:,.2f}")

# Analyze by store type
for store_type in val_features['Type'].unique():
    type_mask = val_features['Type'] == store_type
    if type_mask.sum() > 0:
        type_mae = np.mean(np.abs(val_features.loc[type_mask, 'Weekly_Sales'] - lgb_pred[type_mask]))
        print(f"Store Type {store_type} MAE: {type_mae:,.2f}")

print("\n💾 SAVING RESULTS")
print("=" * 50)

# Save results
results = {
    'Model': ['Naive Seasonal', 'LightGBM'],
    'WMAE': [naive_wmae, lgb_wmae],
    'Records': [len(val_features), len(val_features)]
}

results_df = pd.DataFrame(results)
results_df.to_csv('lightgbm_baseline_results.csv', index=False)

# Save feature importance
importance_df.to_csv('lightgbm_feature_importance.csv', index=False)

# Save predictions
predictions_df = pd.DataFrame({
    'Store': val_features['Store'],
    'Dept': val_features['Dept'],
    'Date': val_features['Date'],
    'Actual': val_features['Weekly_Sales'],
    'IsHoliday': val_features['IsHoliday'],
    'Type': val_features['Type'],
    'Naive_Pred': naive_pred,
    'LightGBM_Pred': lgb_pred,
    'Naive_Error': np.abs(val_features['Weekly_Sales'] - naive_pred),
    'LightGBM_Error': np.abs(val_features['Weekly_Sales'] - lgb_pred)
})
predictions_df.to_csv('lightgbm_predictions.csv', index=False)

print("✅ Results saved:")
print("  - lightgbm_baseline_results.csv")
print("  - lightgbm_feature_importance.csv")
print("  - lightgbm_predictions.csv")

print(f"\n🎯 BASELINE ESTABLISHED!")
print(f"🏆 Target to beat: {lgb_wmae:,.2f} WMAE")
print(f"📈 Next steps: Hyperparameter tuning, feature engineering, ensemble methods")

print("\n🎉 LIGHTGBM BASELINE COMPLETE! 🚀")

🎯 WALMART SALES FORECASTING - LIGHTGBM BASELINE
📥 Loading processed datasets...
Train features shape: (386007, 54)
Validation features shape: (35563, 54)

📊 WMAE METRIC IMPLEMENTATION
✅ WMAE implementation ready

🔧 PREPARING DATA FOR LIGHTGBM
Total features for LightGBM: 50

Missing value analysis:
Features with missing values in train: 14
Features with missing values in validation: 14
Using all training records: 386007
Using all validation records: 35563

🔧 NAIVE BASELINE FOR COMPARISON
🔄 Running naive seasonal baseline...
Naive Seasonal WMAE: 2,142.17

🔧 LIGHTGBM MODEL TRAINING
🔄 Training LightGBM model...
Features: 50
Training samples: 386007
Validation samples: 35563
Training until validation scores don't improve for 50 rounds
[50]	train's l1: 2226.57	valid's l1: 3015.24
[100]	train's l1: 1639.14	valid's l1: 2542.67
Early stopping, best iteration is:
[97]	train's l1: 1646.11	valid's l1: 2525.53
✅ Training completed. Best iteration: 97

📊 MODEL EVALUATION
LightGBM WMAE: 2,418.70
Nai

In [13]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("🎯 WALMART SALES FORECASTING - XGBOOST BASELINE")
print("=" * 60)

# Load processed data
print("📥 Loading processed datasets...")
train_features = pd.read_csv('train_features.csv')
val_features = pd.read_csv('val_features.csv')

# Convert dates back to datetime
train_features['Date'] = pd.to_datetime(train_features['Date'])
val_features['Date'] = pd.to_datetime(val_features['Date'])

print(f"Train features shape: {train_features.shape}")
print(f"Validation features shape: {val_features.shape}")

print("\n📊 WMAE METRIC IMPLEMENTATION")
print("=" * 50)

def wmae(y_true, y_pred, weights):
    """Weighted Mean Absolute Error - Holiday weeks have 5x weight"""
    absolute_errors = np.abs(y_true - y_pred)
    weighted_errors = absolute_errors * weights
    return np.sum(weighted_errors) / np.sum(weights)

def calculate_weights(is_holiday):
    """Calculate weights: 5 for holiday weeks, 1 for normal weeks"""
    return np.where(is_holiday, 5.0, 1.0)

# Custom WMAE objective for XGBoost
def wmae_objective(y_pred, y_true):
    """Custom WMAE objective function for XGBoost"""
    # XGBoost passes DMatrix, need to get labels and weights
    labels = y_true.get_label()
    weights = y_true.get_weight()

    # Calculate gradient (derivative of WMAE)
    residual = y_pred - labels
    grad = weights * np.sign(residual)

    # Calculate hessian (second derivative)
    # For MAE, hessian is 0, but XGBoost needs non-zero values
    hess = weights * 0.01  # Small positive value

    return grad, hess

def wmae_eval(y_pred, y_true):
    """Custom WMAE evaluation function for XGBoost"""
    labels = y_true.get_label()
    weights = y_true.get_weight()

    if weights is None:
        weights = np.ones(len(labels))

    absolute_errors = np.abs(labels - y_pred)
    weighted_errors = absolute_errors * weights
    wmae_score = np.sum(weighted_errors) / np.sum(weights)

    return 'wmae', wmae_score

print("✅ WMAE implementation ready")

print("\n🔧 PREPARING DATA FOR XGBOOST")
print("=" * 50)

# Prepare features (exclude target and identifiers)
exclude_cols = ['Date', 'Weekly_Sales', 'Store', 'Dept']
feature_cols = [col for col in train_features.columns if col not in exclude_cols]

# Handle categorical variables
categorical_cols = ['Type']
for col in categorical_cols:
    if col in feature_cols:
        le = LabelEncoder()
        # Fit on combined data to ensure consistent encoding
        combined_values = pd.concat([train_features[col], val_features[col]]).astype(str)
        le.fit(combined_values)

        train_features[col + '_encoded'] = le.transform(train_features[col].astype(str))
        val_features[col + '_encoded'] = le.transform(val_features[col].astype(str))

        feature_cols.remove(col)
        feature_cols.append(col + '_encoded')

print(f"Total features for XGBoost: {len(feature_cols)}")

# Analyze missing values
print("\nMissing value analysis:")
train_missing = train_features[feature_cols].isnull().sum()
val_missing = val_features[feature_cols].isnull().sum()

print(f"Features with missing values in train: {(train_missing > 0).sum()}")
print(f"Features with missing values in validation: {(val_missing > 0).sum()}")

# XGBoost handles missing values natively, but let's see which features have them
if (train_missing > 0).sum() > 0:
    print("\nFeatures with missing values (train):")
    missing_features = train_missing[train_missing > 0].sort_values(ascending=False)
    for feat, count in missing_features.head(10).items():
        pct = count / len(train_features) * 100
        print(f"  {feat}: {count:,} ({pct:.1f}%)")

print(f"\nUsing all training records: {len(train_features)}")
print(f"Using all validation records: {len(val_features)}")

print("\n🔧 NAIVE BASELINE FOR COMPARISON")
print("=" * 50)

def naive_seasonal_simple(train_df, val_df):
    """Simple seasonal naive with fallbacks"""
    predictions = []

    # Create a mapping of store-dept to recent sales for faster lookup
    recent_sales = {}
    for store in train_df['Store'].unique():
        for dept in train_df['Dept'].unique():
            store_dept_data = train_df[(train_df['Store'] == store) & (train_df['Dept'] == dept)]
            if len(store_dept_data) > 0:
                recent_sales[(store, dept)] = store_dept_data['Weekly_Sales'].iloc[-4:].mean()  # Last 4 weeks avg

    overall_mean = train_df['Weekly_Sales'].mean()

    for _, row in val_df.iterrows():
        store, dept = row['Store'], row['Dept']

        if (store, dept) in recent_sales:
            pred = recent_sales[(store, dept)]
        else:
            pred = overall_mean

        predictions.append(pred)

    return np.array(predictions)

# Run naive baseline
print("🔄 Running naive seasonal baseline...")
naive_pred = naive_seasonal_simple(train_features, val_features)
val_weights = calculate_weights(val_features['IsHoliday'].values)
naive_wmae = wmae(val_features['Weekly_Sales'].values, naive_pred, val_weights)

print(f"Naive Seasonal WMAE: {naive_wmae:,.2f}")

print("\n🔧 XGBOOST MODEL TRAINING")
print("=" * 50)

# XGBoost parameters - more conservative than LightGBM
xgb_params = {
    'objective': 'reg:absoluteerror',  # MAE objective
    'eval_metric': 'mae',
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 5,
    'reg_alpha': 0.1,  # L1 regularization
    'reg_lambda': 0.1,  # L2 regularization
    'random_state': 42,
    'tree_method': 'hist',  # Faster for large datasets
    'verbosity': 0
}

print("🔄 Training XGBoost model...")
print(f"Features: {len(feature_cols)}")
print(f"Training samples: {len(train_features)}")
print(f"Validation samples: {len(val_features)}")

# Prepare data for XGBoost
X_train = train_features[feature_cols]
y_train = train_features['Weekly_Sales']
X_val = val_features[feature_cols]
y_val = val_features['Weekly_Sales']

# Calculate weights for WMAE
train_weights = calculate_weights(train_features['IsHoliday'].values)

# Create DMatrix objects
dtrain = xgb.DMatrix(X_train, label=y_train, weight=train_weights)
dval = xgb.DMatrix(X_val, label=y_val, weight=val_weights)

# Train model with early stopping
print("Training with weighted MAE objective...")
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, 'train'), (dval, 'eval')],
    early_stopping_rounds=50,
    verbose_eval=50,
    feval=wmae_eval  # Custom WMAE evaluation
)

print(f"✅ Training completed. Best iteration: {xgb_model.best_iteration}")

print("\n📊 MODEL EVALUATION")
print("=" * 50)

# Make predictions
xgb_pred = xgb_model.predict(dval)

# Calculate WMAE
xgb_wmae = wmae(val_features['Weekly_Sales'].values, xgb_pred, val_weights)

print(f"XGBoost WMAE: {xgb_wmae:,.2f}")
print(f"Naive WMAE: {naive_wmae:,.2f}")
print(f"Improvement: {((naive_wmae - xgb_wmae) / naive_wmae * 100):+.1f}%")

print("\n📈 FEATURE IMPORTANCE")
print("=" * 50)

# Get feature importance
importance_dict = xgb_model.get_score(importance_type='gain')
importance_df = pd.DataFrame({
    'feature': list(importance_dict.keys()),
    'importance': list(importance_dict.values())
})
importance_df = importance_df.sort_values('importance', ascending=False)

print("Top 15 most important features:")
top_features = importance_df.head(15)
for i, (_, row) in enumerate(top_features.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:>10,.0f}")

print("\n📊 PREDICTION ANALYSIS")
print("=" * 50)

# Analyze predictions by holiday vs non-holiday
holiday_mask = val_features['IsHoliday'] == True
non_holiday_mask = val_features['IsHoliday'] == False

if holiday_mask.sum() > 0:
    holiday_mae = np.mean(np.abs(val_features.loc[holiday_mask, 'Weekly_Sales'] - xgb_pred[holiday_mask]))
    print(f"Holiday weeks MAE: {holiday_mae:,.2f}")

if non_holiday_mask.sum() > 0:
    non_holiday_mae = np.mean(np.abs(val_features.loc[non_holiday_mask, 'Weekly_Sales'] - xgb_pred[non_holiday_mask]))
    print(f"Non-holiday weeks MAE: {non_holiday_mae:,.2f}")

# Analyze by store type
for store_type in val_features['Type'].unique():
    type_mask = val_features['Type'] == store_type
    if type_mask.sum() > 0:
        type_mae = np.mean(np.abs(val_features.loc[type_mask, 'Weekly_Sales'] - xgb_pred[type_mask]))
        print(f"Store Type {store_type} MAE: {type_mae:,.2f}")

print("\n💾 SAVING RESULTS")
print("=" * 50)

# Save results
results = {
    'Model': ['Naive Seasonal', 'XGBoost'],
    'WMAE': [naive_wmae, xgb_wmae],
    'Records': [len(val_features), len(val_features)]
}

results_df = pd.DataFrame(results)
results_df.to_csv('xgboost_baseline_results.csv', index=False)

# Save feature importance
importance_df.to_csv('xgboost_feature_importance.csv', index=False)

# Save predictions
predictions_df = pd.DataFrame({
    'Store': val_features['Store'],
    'Dept': val_features['Dept'],
    'Date': val_features['Date'],
    'Actual': val_features['Weekly_Sales'],
    'IsHoliday': val_features['IsHoliday'],
    'Type': val_features['Type'],
    'Naive_Pred': naive_pred,
    'XGBoost_Pred': xgb_pred,
    'Naive_Error': np.abs(val_features['Weekly_Sales'] - naive_pred),
    'XGBoost_Error': np.abs(val_features['Weekly_Sales'] - xgb_pred)
})
predictions_df.to_csv('xgboost_predictions.csv', index=False)

print("✅ Results saved:")
print("  - xgboost_baseline_results.csv")
print("  - xgboost_feature_importance.csv")
print("  - xgboost_predictions.csv")

print(f"\n🎯 BASELINE ESTABLISHED!")
print(f"🏆 Target to beat: {xgb_wmae:,.2f} WMAE")

# Compare with LightGBM if available
try:
    lgb_results = pd.read_csv('lightgbm_baseline_results.csv')
    lgb_wmae = lgb_results[lgb_results['Model'] == 'LightGBM']['WMAE'].iloc[0]
    print(f"\n🔍 MODEL COMPARISON:")
    print(f"XGBoost WMAE: {xgb_wmae:,.2f}")
    print(f"LightGBM WMAE: {lgb_wmae:,.2f}")
    if xgb_wmae < lgb_wmae:
        improvement = ((lgb_wmae - xgb_wmae) / lgb_wmae * 100)
        print(f"🏆 XGBoost wins by {improvement:+.1f}%")
    else:
        improvement = ((xgb_wmae - lgb_wmae) / xgb_wmae * 100)
        print(f"🏆 LightGBM wins by {improvement:+.1f}%")
except:
    print("📝 Note: Run LightGBM baseline to compare models")

print(f"\n📈 Next steps: Hyperparameter tuning, feature engineering, ensemble methods")
print("\n🎉 XGBOOST BASELINE COMPLETE! 🚀")

🎯 WALMART SALES FORECASTING - XGBOOST BASELINE
📥 Loading processed datasets...
Train features shape: (386007, 54)
Validation features shape: (35563, 54)

📊 WMAE METRIC IMPLEMENTATION
✅ WMAE implementation ready

🔧 PREPARING DATA FOR XGBOOST
Total features for XGBoost: 50

Missing value analysis:
Features with missing values in train: 14
Features with missing values in validation: 14

Features with missing values (train):
  Weekly_Sales_lag_52: 159,891 (41.4%)
  Weekly_Sales_lag_12: 38,529 (10.0%)
  Weekly_Sales_lag_8: 25,919 (6.7%)
  Weekly_Sales_lag_4: 13,109 (3.4%)
  Weekly_Sales_lag_2: 6,609 (1.7%)
  Weekly_Sales_rolling_std_4: 6,609 (1.7%)
  Weekly_Sales_rolling_std_26: 6,609 (1.7%)
  Weekly_Sales_rolling_std_8: 6,609 (1.7%)
  Weekly_Sales_rolling_std_12: 6,609 (1.7%)
  Weekly_Sales_lag_1: 3,322 (0.9%)

Using all training records: 386007
Using all validation records: 35563

🔧 NAIVE BASELINE FOR COMPARISON
🔄 Running naive seasonal baseline...
Naive Seasonal WMAE: 2,142.17

🔧 XGBOOST